<a href="https://colab.research.google.com/github/Interpause/pseudo-text/blob/master/how_to_query_a_database_EX_edition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
%matplotlib inline
%unload_ext google.colab.data_table
%tensorflow_version 2.x
from google.colab import drive #drive.flush_and_unmount()
import os
import numpy as np
import pandas as pd
from collections import OrderedDict
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/CSIT Internship/code')

The google.colab.data_table extension is not loaded.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
df = pd.read_pickle('./raw/myNewsFinal.pkl')
#df1.join(df2, how='outer').sort_index(1)

In [0]:
def query(sent,vectorizer,dist,embeds,index,db,top_n=10,reverse=False):
    aquery = vectorizer(sent)
    reply = OrderedDict(sorted(zip(index,dist(aquery,embeds)),key = lambda x: x[1],reverse=reverse))
    for ind,dist in list(reply.items())[:10]:
        print('%s\t%f\t%s'%(str(ind)[:4],dist,db.df.loc[df.uuid==ind].title.values[0]))
    print(df.loc[df.uuid==list(reply.keys())[0]].text.values[0])

In [0]:
#@title TS_SS stuff {display-mode: "form"}
#@markdown https://github.com/taki0112/Vector_Similarity (i stole this)
class TS_SS:
    def Cosine(self, vec1: np.ndarray, vec2: np.ndarray):
        return np.dot(vec1, vec2.T)/(np.linalg.norm(vec1) * np.linalg.norm(vec2))

    def VectorSize(self, vec: np.ndarray):
        return np.linalg.norm(vec)

    def Euclidean(self, vec1: np.ndarray, vec2: np.ndarray):
        return np.linalg.norm(vec1-vec2)

    def Theta(self, vec1: np.ndarray, vec2: np.ndarray):
        return np.arccos(self.Cosine(vec1, vec2)) + np.radians(10) #apparently the "best" value to solve overlapping problem (seems arbitrary tho)

    def Triangle(self, vec1: np.ndarray, vec2: np.ndarray):
        theta = np.radians(self.Theta(vec1, vec2))
        return (self.VectorSize(vec1) * self.VectorSize(vec2) * np.sin(theta))/2

    def Magnitude_Difference(self, vec1: np.ndarray, vec2: np.ndarray):
        return abs(self.VectorSize(vec1) - self.VectorSize(vec2))

    def Sector(self, vec1: np.ndarray, vec2: np.ndarray):
        ED = self.Euclidean(vec1, vec2)
        MD = self.Magnitude_Difference(vec1, vec2)
        theta = self.Theta(vec1, vec2)
        return np.pi * (ED + MD)**2 * theta/360

    def __call__(self, vec1: np.ndarray, vec2: np.ndarray):
        return self.Triangle(vec1, vec2) * self.Sector(vec1, vec2)
tsss = TS_SS()

In [0]:
import sklearn.metrics
def cosine(vec1,samples):
    ans = sklearn.metrics.pairwise.cosine_similarity(np.array([vec1]),samples)
    return 1 - ans[0]

# Future approaches
- BERTsumEXT sentences before embedding
- Factor in date

##TODO:
- word2vec weighted by tfidf
- bertEXT instead of textRank
- gensim sent vec, doc vec instead of sent-transformers
- Try out flair's stacked embeddings (and use tfidf weighting o_o) (remove stopwords after embedding step; likewise remove stopword vectors) (adding tfidf might be overkill if further training RNN on top)
- somehow deal with word sense disambiguation
- spacy's NER? flair's NER? well it will be needed for active scrape
- NER based search approach

#TFIDF Base

In [9]:
import string
import spacy
import nltk
from collections import OrderedDict
nltk.download('stopwords')
removables = nltk.corpus.stopwords.words('english')
replacements = OrderedDict({'barack obama':'obama','donald trump':'trump','health care':'healthcare','hillary clinton':'hillary','president obama':'obama','barack':'obama','donald':'trump','gov ':'government ','said':'say'})
class LemmaTokenizer(object):
    def __init__(self):
        self.nlp = spacy.load('en', disable=['parser', 'ner'])
    def __call__(self, articles):
        for rep,val in replacements.items(): articles = articles.replace(rep,val)
        return [str(t.lemma_) for t in self.nlp(articles) if t.text not in removables]
lemma = LemmaTokenizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import OrderedDict
text_vectorizer = TfidfVectorizer(lowercase=True,
                            #tokenizer=lemma, #lemmatizer somehow accents word sense disambiguation problems
                            stop_words='english',
                            ngram_range=(1,2),
                            max_features=25000,
                            norm=None)
title_vectorizer = TfidfVectorizer(lowercase=True,
                            #tokenizer=lemma, #lemmatizer somehow accents word sense disambiguation problems
                            stop_words='english',
                            ngram_range=(1,2),
                            max_features=25000,
                            norm=None)
#TODO, increase ngram_range since word2vecs support phrases (underscores i think?)

In [0]:
database = pd.concat({'df':df.copy(),
                 'tfidf_text':pd.DataFrame.sparse.from_spmatrix(text_vectorizer.fit_transform(df.text),index=df.index),
                 'tfidf_title':pd.DataFrame.sparse.from_spmatrix(title_vectorizer.fit_transform(df.title),index=df.index)
                 },
                axis=1)#,copy=False)

In [130]:
test_query = 'vaccines cause autism'
print('Cosine text')
query(test_query,
      lambda s: text_vectorizer.transform([s]).toarray()[0],
      cosine,
      database.tfidf_text.values,
      database.df.uuid,
      database)
print('\nTS-SS text')
query(test_query,
      lambda s: text_vectorizer.transform([s]).toarray()[0],
      tsss,
      database.tfidf_text.values,
      database.df.uuid,
      database)
print('\nCosine title')
query(test_query,
      lambda s: title_vectorizer.transform([s]).toarray()[0],
      cosine,
      database.tfidf_title.values,
      database.df.uuid,
      database)
print('\nTS-SS title')
query(test_query,
      lambda s: title_vectorizer.transform([s]).toarray()[0],
      tsss,
      database.tfidf_title.values,
      database.df.uuid,
      database)

Cosine text
245b	0.607406	I'm Thankful for Autism, Eight Reasons this Thanksgiving I Am Giving Thanks
ba97	0.652245	Autism and ADHD share genes
aeb0	0.660046	Vaccines Fast Facts
946e	0.729204	A Parent-To-Parent Campaign To Get Vaccine Rates Up
ecfd	0.729218	Her son died. And then anti-vaxers attacked her
56cf	0.741302	Autism News
7a57	0.774695	Autism News
14a6	0.809315	Samoa measles deaths rise above 50, mostly small children
6aa5	0.809315	Samoa measles deaths rise above 50, mostly small children
312a	0.814439	Samoa measles deaths rise above 50, mostly small children
I never really thought I would utter the words "I'm thankful for autism" though I have and while I certainly have moments of self-doubt, feelings of being overwhelmed, wants of wishing life was easier for my daughter, there are so many things that I am thankful for, because of autism. First, autism has allowed me to gain true friends. Choosing to be a friend to a person or with a family who lives with a loved one on the sp

## Brief analysis
TS-SS seems better than cosine on average. (probably not confirmation bias)

In [0]:
import pickle
with open('./tfidf_vectorizer.bin','wb') as f:
    pickle.dump({
        'tfidf_text':text_vectorizer,
        'tfidf_title':title_vectorizer
    },f)

# Reload Vectorizers

In [0]:
import pickle
with open('./tfidf_vectorizer.bin','rb') as f:
    temp = pickle.load(f)
    text_vectorizer = temp['tfidf_text']
    title_vectorizer = temp['tfidf_title']
    del temp

In [0]:
database = pd.concat({'df':df.copy(),
                 'tfidf_text':pd.DataFrame.sparse.from_spmatrix(text_vectorizer.transform(df.text),index=df.index),
                 'tfidf_title':pd.DataFrame.sparse.from_spmatrix(title_vectorizer.transform(df.title),index=df.index)
                 },
                axis=1)#,copy=False)

# Word embed + WMD

In [0]:
fit_embed(glove,database.tfidf_title.values,title_vectorizer.vocabulary_,database.df.uuid)
#!pip install wmd
#probably only works for tfidf plus wmd
from wmd import WMD
def fit_wmd(word_embeds,doc_embeds,vocab_ind,uuids):
    ind_embed = {i:word_embeds[w] for w,i in vocab_ind.items()}
    nbow = {uuid:(uuid,tuple(range(len(vocab_ind))),doc_embeds[i]) for i,uuid in enumerate(uuids)}
    return WMD(ind_embed, nbow)

def query_wmd(sent,wmd):
    return wmd.nearest_neighbors(sent)

# Sent2Vec Base

In [0]:
#!pip install -U sentence-transformers
promising = (
'bert-base-wikipedia-sections-mean-tokens', #crap
'distilbert-base-nli-stsb-mean-tokens',
'roberta-base-nli-stsb-mean-tokens',
'roberta-base-nli-mean-tokens', #crap
'distilbert-base-nli-mean-tokens'
)

In [43]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

100%|██████████| 245M/245M [00:10<00:00, 22.6MB/s]


In [0]:
from gensim.summarization import summarize, keywords
def encode_document(sent,model):
    return model.encode([sent])[0]
    #try:
        #ext = summarize(sent,split=True)
        #if len(ext) <= 1:
        #    return model.encode([sent])[0]
        #else:
        #    return np.mean(model.encode(ext),axis=0)
    #except ValueError:
    #    return model.encode([sent])[0]
def encode_documents(sents,model):
    return np.stack(pd.Series(sents).apply(encode_document,args=(model,)))
#np.mean(model.encode(summarize(df.loc[31].text,split=True)),axis=0)

In [107]:
all_enc = {}
for model_name in promising:
    model = SentenceTransformer(model_name)
    all_enc[model_name+'_title'] = pd.DataFrame(encode_documents(df.title,model),index=df.index)
    all_enc[model_name+'_text'] = pd.DataFrame(encode_documents(df.text,model),index=df.index)

100%|██████████| 459M/459M [00:21<00:00, 21.7MB/s]
100%|██████████| 459M/459M [00:20<00:00, 22.4MB/s]
100%|██████████| 245M/245M [00:11<00:00, 20.8MB/s]


In [0]:
import pickle
with open('./raw/sentTransData.bin','wb') as f:
    pickle.dump(all_enc,f)

In [0]:
import pickle
with open('./raw/sentTransData.bin','rb') as f:
    enc = pickle.load(f)

In [0]:
db2 = pd.concat(all_enc,axis=1)#,copy=False)

In [131]:
test_query = "vaccines cause autism"
for model_name in promising:
    model = SentenceTransformer(model_name)
    print(model_name)
    print('Title')
    query(test_query,
        lambda s: encode_document(s,model),
        tsss,
        db2[model_name+'_title'].values,
        db2.df.uuid,
        db2)
    print('\nText')
    query(test_query,
        lambda s: encode_document(s,model),
        tsss,
        db2[model_name+'_text'].values,
        db2.df.uuid,
        db2)
    print('\n')

bert-base-wikipedia-sections-mean-tokens
Title
255a	1195552.625000	Prenatal and early life exposure to multiple air pollutants increases odds of toddler allergies: Early exposure to candles, cats and environmental tobacco smoke showed greatest impact
cfb0	1195573.125000	Eating more gluten early in life is tied to children's higher risk of celiac disease
fd63	1195573.125000	Health experts warn of emerging threat of Nipah virus
8de7	1195573.125000	Health experts warn of emerging threat of Nipah virus
527a	1195583.000000	'Junk DNA' affects inherited cancer risk
d846	1195587.875000	Inbreeding and population/demographic shifts could have led to Neanderthal extinction: Neanderthal extinction could have occurred without environmental pressure or competition with modern humans
78a1	1195595.375000	New vaccine will stop the spread of bovine TB
2981	1195596.000000	There's A Promising New Vaccine For One Of The World's Top Health Threats
eb65	1195599.250000	Why The Ebola Response Is A Success In G

## Brief Analysis
Actually loses out to tfidf based on this search query. Seems more capable than tfidf at gathered articles that share similar contexts - except they are too broad. For example, using the PIE expressway query, it doesn't pull up results related to pie crusts. Corroborates that a tfidf weighted approach might be best.

# GENSIM
used for word embeddings (tfidf as weights approach), provides distance formulae like cosine and WMD

In [0]:
import gensim
from gensim.models import KeyedVectors

In [0]:
glove_loc = '/content/drive/My Drive/CSIT Internship/code/gensim_glove_vectors.txt' #Common crawl uncased 42 billion tokens 300 dimensions
word2vec_loc = '/content/drive/My Drive/CSIT Internship/code/GoogleNews-vectors-negative300.bin' #google news 3 million tokens 300 dimensions

In [0]:
newsve = KeyedVectors.load_word2vec_format(word2vec_loc, binary=True)
#glove = KeyedVectors.load_word2vec_format(glove_loc, mmap='r')
#word_vectors.distance("media", "media") #cosine

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


KeyboardInterrupt: ignored